In [76]:
import pandas as pd
import os
import warnings
from fuzzywuzzy import fuzz
from collections import defaultdict
import numpy as np

warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

In [3]:
work_dir = 'sample_data'

In [124]:
code_to_type = {1:'individual', 2:'organization'}
concatted_dfs = []
for filename in ['npidata', 'endpoint', 'othername', 'pl']:
    file = [f for f in os.listdir(work_dir) if filename in f][0]
    file_df = pd.read_csv(os.path.join(work_dir,file))
    if filename == 'npidata':
        npis_with_types = file_df[['NPI', 'Entity Type Code']]
    else:
        file_df = file_df.merge(npis_with_types, on = 'NPI')
    field_list = []
    for code in code_to_type.keys():
        subset = file_df.loc[file_df['Entity Type Code']==code].dropna(axis=1, how = 'all')
        transposed_fields = subset.T.reset_index(drop=False).rename(columns = {'index':'original_field'})[['original_field']]
        transposed_fields[code_to_type[code]] = True
        if filename != 'npidata':
            transposed_fields = transposed_fields.loc[~(transposed_fields['original_field']=='Entity Type Code')]
        transposed_fields.set_index('original_field', inplace=True)
        field_list.append(transposed_fields)
    concatted_df = pd.concat(field_list, axis=1)
    concatted_df['file'] = filename
    concatted_dfs.append(concatted_df)
fields_for_all_files = pd.concat(concatted_dfs)
fields_for_all_files['data_element'] = list(pd.Series(fields_for_all_files.index).apply(lambda x: x.replace('Provider','').replace('Healthcare','').replace(' Business','').replace('Other','').split('(')[0].replace(' - ',' ').strip().split('_')[0].replace(' ','_').replace('__','_').lower().replace('_one','_1').replace('_two','_2')))
address_types = [de.replace('_postal_code','') for de in fields_for_all_files['data_element'] if 'postal_code' in de and de!='postal_code']
fields_for_all_files['address_type'] = fields_for_all_files['data_element'].apply(lambda x: max([at for at in address_types if at in x], default=''))
fields_for_all_files['data_element'] = [row['data_element'].replace(row["address_type"],'').rstrip('_-').lstrip('_-') for x,row in fields_for_all_files.iterrows()]


In [126]:
fields_for_all_files.sort_values('data_element').to_csv('fields.csv')

In [111]:
fields_for_all_files.loc[(fields_for_all_files['data_element'].apply(lambda x: 'name' in x))&(fields_for_all_files['organization'])]

,individual,organization,file,data_element,address_type
original_field,,,,,
Provider Business Mailing Address City Name,True,True,npidata,city_name,mailing_address
Provider Business Mailing Address State Name,True,True,npidata,state_name,mailing_address
Provider Business Practice Location Address City Name,True,True,npidata,city_name,practice_location_address
Provider Business Practice Location Address State Name,True,True,npidata,state_name,practice_location_address
Provider Organization Name (Legal Business Name),NaN,True,npidata,organization_name,
Provider Other Organization Name,NaN,True,npidata,other_organization_name,
Provider Other Organization Name Type Code,NaN,True,npidata,other_organization_name_type_code,
Authorized Official Last Name,NaN,True,npidata,authorized_official_last_name,
Authorized Official First Name,NaN,True,npidata,authorized_official_first_name,
